In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/bigdata/airbnb

/content/drive/MyDrive/Colab Notebooks/bigdata/airbnb


In [ ]:
df = pd.read_csv('data/train_users_2.csv')
df = df.fillna(-1)

In [ ]:
df.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination'],
      dtype='object')

In [ ]:
df = df.drop(['id', 'date_first_booking'], axis = 1)

## 轉換格式

In [ ]:
#Converting below columns as categories for plotting in graphs
cate_col = ['affiliate_channel',
             'affiliate_provider',
             'first_affiliate_tracked',
             'first_device_type',
             'gender',
             'language',
             'signup_app',
             'signup_method',
             'signup_flow',
            'country_destination']

for i in cate_col:
    df[i] = df[i].astype('category')
    
df['date_account_created'] = pd.to_datetime(df['date_account_created'])
df['timestamp_first_active'] = pd.to_datetime(df['timestamp_first_active'], format='%Y%m%d%H%M%S')

## 去除目的地國家=='NDF'

In [ ]:
print(f'原始df形狀(包括預測目標column): {df.shape}')
s = sum(df['country_destination'] == 'NDF')
print(f'目的地國家=NDF數量: {s}')

原始df形狀(包括預測目標column): (213451, 14)
目的地國家=NDF數量: 124543


In [ ]:
df = df.loc[df['country_destination'] != 'NDF', :].reset_index(drop = True)
df.shape

(88908, 14)

## 特徵工程

In [ ]:
df.isnull().sum()

date_account_created       0
timestamp_first_active     0
gender                     0
age                        0
signup_method              0
signup_flow                0
language                   0
affiliate_channel          0
affiliate_provider         0
first_affiliate_tracked    0
signup_app                 0
first_device_type          0
first_browser              0
country_destination        0
dtype: int64

* 產生 'date_account_created', 'timestamp_first_active'的year, month項

In [ ]:
dac = np.vstack(df['date_account_created'].astype(str).str.replace(' 00:00:00', '').apply(lambda x: list(map(int, x.split('-')))).values)
df['dac_year'] = dac[:,0]
df['dac_month'] = dac[:,1]
df = df.drop(['date_account_created'], axis=1)

In [ ]:
tfa = np.vstack(df['timestamp_first_active'].astype(str).str[:10].apply(lambda x: list(map(int, x.split('-')))).values)
df['tfa_year'] = tfa[:, 0]
df['tfa_month'] = tfa[:, 1]
df = df.drop(['timestamp_first_active'], axis=1)

In [ ]:
df.shape

(88908, 16)

In [ ]:
df[['dac_year', 'dac_month', 'tfa_year', 'tfa_month']].head()

,dac_year,dac_month,tfa_year,tfa_month
0,2010,9,2009,6
1,2011,12,2009,10
2,2010,9,2009,12
3,2010,1,2010,1
4,2010,1,2010,1


* 處理太多類別的category columns(first_browser有40類太多了，而且很多類別的數量只有1個)

In [ ]:
cate = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel',
             'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in cate:
    print(f, '共有', df[f].value_counts().shape[0], '類')

gender 共有 4 類
signup_method 共有 3 類
signup_flow 共有 17 類
language 共有 25 類
affiliate_channel 共有 8 類
affiliate_provider 共有 18 類
first_affiliate_tracked 共有 8 類
signup_app 共有 4 類
first_device_type 共有 9 類
first_browser 共有 40 類


In [ ]:
df['first_browser'].value_counts().sort_values(0)

Avant Browser               1
SlimBrowser                 1
NetNewsWire                 1
Kindle Browser              1
CometBird                   1
Stainless                   1
Palm Pre web browser        1
Mozilla                     1
Googlebot                   1
Yandex.Browser              1
TheWorld Browser            2
Opera Mini                  2
CoolNovo                    3
wOSBrowser                  3
SeaMonkey                   4
Pale Moon                   4
TenFourFox                  4
SiteKiosk                   7
IE Mobile                   7
Camino                      7
IceWeasel                   8
Sogou Explorer              9
Iron                        9
RockMelt                   12
Mobile Firefox             13
Apple Mail                 15
Maxthon                    16
BlackBerry Browser         22
Silk                       43
Chromium                   43
AOL Explorer               63
Opera                      64
Android Browser           280
Chrome Mob

In [ ]:
# 將數量小於8個以內的類別改成others
sort_fb = df['first_browser'].value_counts().sort_values(0)
others = list(sort_fb[sort_fb.values < 8].index)
len(others), others

(20,
 ['Avant Browser',
  'SlimBrowser',
  'NetNewsWire',
  'Kindle Browser',
  'CometBird',
  'Stainless',
  'Palm Pre web browser',
  'Mozilla',
  'Googlebot',
  'Yandex.Browser',
  'TheWorld Browser',
  'Opera Mini',
  'CoolNovo',
  'wOSBrowser',
  'SeaMonkey',
  'Pale Moon',
  'TenFourFox',
  'SiteKiosk',
  'IE Mobile',
  'Camino'])

In [ ]:
df['first_browser'][df['first_browser'].isin(others)] = 'others'
df['first_browser'] = df['first_browser'].astype('category')

In [ ]:
df['first_browser'].value_counts().sort_values(0)

IceWeasel                 8
Sogou Explorer            9
Iron                      9
RockMelt                 12
Mobile Firefox           13
Apple Mail               15
Maxthon                  16
BlackBerry Browser       22
Silk                     43
Chromium                 43
others                   53
AOL Explorer             63
Opera                    64
Android Browser         280
Chrome Mobile           408
Mobile Safari          7181
-unknown-              7618
IE                     7723
Firefox               15730
Safari                19721
Chrome                29877
Name: first_browser, dtype: int64

* 處理年齡離群值(小於16歲(未成年)和大於100歲的改成-1)

In [ ]:
age = df['age'].values
df['age'] = np.where(np.logical_or(age < 10, age > 100), -1, age)

## 儲存資料

In [ ]:
df.to_csv('airbnb_edit.csv', index = False)